<a href="https://colab.research.google.com/github/sonosergi/Workout-Classifier/blob/main/WorkOutClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### MOUNT GOOGLE DRIVE

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### IMPORT MODULES


In [5]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import zipfile
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import string
import urllib.request
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [3]:
from datetime import datetime
from collections import deque
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import InceptionResNetV2

### IMPORT DATA

In [ ]:
# Set the file path of the downloaded zip file from Kaggle
url = '/file path downloaded from kaggle/archive.zip'

# Open the zip file using zipfile.ZipFile
zip_ref = zipfile.ZipFile(url, "r")

# Extract all files from the zip file to the specified directory (/tmp/Data-Keggle)
zip_ref.extractall("/tmp/Data-Keggle")
zip_ref.close() 


### HYPERPARAMETER

In [ ]:
height = 256
width = 256
channels = 3
BATCH_SIZE = 32
SEED = 127
img_shape = (height, width, channels)
IMG_SIZE = (height, width)
CLASSES = 22

### PROCESS DATA

In [ ]:
# Set the path to the directory where the dataset is stored
DATA_DIR = '/tmp/Data-Keggle'

# Split the dataset into training (80%) and validation (20%) sets using Keras' image_dataset_from_directory function
# The labels are inferred from the directory structure
# The images are resized to IMG_SIZE
# The data is shuffled and divided into batches of size BATCH_SIZE
# A random seed is set for reproducibility
training_data = tf.keras.utils.image_dataset_from_directory(DATA_DIR,
  labels = 'inferred',
  label_mode = 'categorical',
  validation_split = 0.3,
  subset = 'training',
  image_size = IMG_SIZE,
  shuffle = True,
  batch_size = BATCH_SIZE,
  seed = 127
)

# Create the validation dataset using the same settings as the training dataset, but with a different subset parameter
test_data = tf.keras.utils.image_dataset_from_directory(DATA_DIR,
  labels = 'inferred',
  label_mode = 'categorical',
  validation_split = 0.3,
  subset = 'validation',
  image_size = IMG_SIZE,
  shuffle = True,
  batch_size = BATCH_SIZE,
  seed = 127
)

# Save the class names to a file for later reference
labels = training_data.class_names
print(labels)

with open('workout_label.txt', 'w') as f:
    for workout_class in labels:
        f.write(f'{workout_class}\n')


In [ ]:
#training_data = ImageDataGenerator(rescale = 1/255)
#test_data = ImageDataGenerator(rescale = 1/255)

### DATA AUGMENTATION

In [ ]:
# Define a data augmentation pipeline using the Keras Sequential API
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal"),   # randomly flip images horizontally
  tf.keras.layers.RandomRotation(0.2),        # randomly rotate images up to 20%
  tf.keras.layers.RandomZoom(0.2),            # randomly zoom images up to 20%
  tf.keras.layers.RandomContrast(0.2),        # randomly adjust contrast up to 20%
  tf.keras.layers.GaussianNoise(0.2),         # add random Gaussian noise to images with a standard deviation of 0.2
  tf.keras.layers.RandomBrightness(0.2)       # randomly adjust brightness up to 20%
])

# Apply the data augmentation pipeline to the training data using the map() method
training_data = training_data.map(lambda x, y: (data_augmentation(x, training=True), y))


### DATA VISUALIZATION

In [ ]:
# Define a function to display images from a TensorFlow dataset
def show_img(data):
    # Set the size of the figure
    plt.figure(figsize=(10,10))
    # Take the first batch of images and labels from the dataset
    for images, labels in data.take(1):
        # Loop through the first 9 images in the batch
        for i in range(9):
            # Create a subplot for each image
            ax = plt.subplot(3, 3, i + 1)
            # Display the image as a numpy array with uint8 data type
            ax.imshow(images[i].numpy().astype("uint8"))
            # Turn off the axis labels for the subplot
            ax.axis("off")

In [ ]:
show_img(training_data)

### MODEL

In [ ]:
# Import the InceptionResNetV2 pre-trained model from Keras with weights pre-trained on ImageNet
pre_trained_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=img_shape, pooling='avg')

# Set all layers in the pre-trained model to non-trainable
for layer in pre_trained_model.layers:
    layer.trainable = False

# Print a summary of the pre-trained model
pre_trained_model.summary()


### FREEZE THE MODEL

In [ ]:
# Set the output of the pre-trained model as the input for the new layers
x = pre_trained_model.output

# Add a batch normalization layer to normalize the input to the next layer
x = tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)

# Add a dropout layer to randomly drop out 20% of the input units
x = tf.keras.layers.Dropout(0.2)(x)

# Add a fully connected layer with 1024 units and ReLU activation function
x = tf.keras.layers.Dense(1024, activation='relu')(x)

# Add a fully connected layer with 512 units and ReLU activation function
x = tf.keras.layers.Dense(512, activation='relu')(x)

# Add a fully connected layer with 256 units and ReLU activation function
x = tf.keras.layers.Dense(256, activation='relu')(x)

# Add a dropout layer to randomly drop out 20% of the input units
x = tf.keras.layers.Dropout(0.2)(x)

# Add a fully connected layer with the number of units equal to the number of labels and softmax activation function
predictions = tf.keras.layers.Dense(len(labels), activation='softmax')(x)

# Create a new Keras model with the pre-trained model's input as the input and the new layers as the output
iTrainer = tf.keras.models.Model(inputs=pre_trained_model.input, outputs=predictions)

# Compile the model with Adam optimizer, categorical cross-entropy loss, and accuracy metric
iTrainer.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

# Print a summary of the model's architecture and parameters
iTrainer.summary()

### TRAIN

In [ ]:
# Define callbacks to save the best model and stop training early if accuracy stops improving
checkpoint_callback = ModelCheckpoint('/your/path/best_model.h5',
                                       monitor='val_accuracy',
                                       verbose=1,
                                       save_best_only=True,
                                       mode='max')

early_stopping_callback = EarlyStopping(monitor='accuracy',
                                        patience=5,
                                        mode='auto',
                                        restore_best_weights=True)

# Train the model with the defined callbacks
history = iTrainer.fit(training_data,
                       validation_data=test_data,
                       epochs=200,
                       callbacks=[early_stopping_callback, checkpoint_callback])


### CONVERT THE .h5 FILE TO .json

In [ ]:
!pip install tensorflowjs

In [ ]:
!tensorflowjs_converter --input_format=keras /your/path/best_model.h5 /your/path